In [ ]:
# install.packages("tidyverse")

### 1. Load data

In [ ]:
library(tidyverse)
library(readxl)
options(repr.plot.width=8, repr.plot.height=6)

path <- list.files(".", pattern=".xlsx")
all <- read_excel(path, sheet=1) # load summary data

### 2. Plotting
#### a. Abundance

In [ ]:
all %>%
    filter(population != "unknown" & population != "beads") %>%
    group_by(treatment, experiments, population) %>%
    summarize(sd=sd(abundance),
              avg=mean(abundance)) %>%
    ggplot(aes(experiments, avg, group=treatment, fill=treatment)) +
    geom_bar(stat="identity", position=position_dodge()) +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +
    ylab("Abundance (cells uL-1)")

#### b. Scatter

In [ ]:
all %>%
    filter(population != "unknown" & population != "beads") %>%
    group_by(treatment, experiments, population) %>%
    summarize(sd=sd(scatter),
              avg=mean(scatter)) %>%
    ggplot(aes(experiments, avg, group=treatment, fill=treatment)) +
    geom_bar(stat="identity", position=position_dodge()) +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +
    ylab("Scatter (normalized to beads)")

#### c. Red fluorescence

In [ ]:
all %>%
    filter(population != "unknown" & population != "beads") %>%
    group_by(treatment, experiments, population) %>%
    summarize(sd=sd(red),
              avg=mean(red)) %>%
    ggplot(aes(experiments, avg, group=treatment, fill=treatment)) +
    geom_bar(stat="identity", position=position_dodge()) +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +

    ylab("Red fluo (normalized to beads)")

#### d. Orange fluorescence

In [ ]:
all %>%
    filter(population != "unknown" & population != "beads") %>%
    group_by(treatment, experiments, population) %>%
    summarize(sd=sd(orange),
              avg=mean(orange)) %>%
    ggplot(aes(experiments, avg, group=treatment, fill=treatment)) +
    geom_bar(stat="identity", position=position_dodge()) +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +
    ylab("Orange fluo (normalized to beads)")

#### e. Red fluorescence, normalized to scatter

In [ ]:
all %>%
    filter(population != "unknown" & population != "beads") %>%
    group_by(treatment, experiments, population) %>%
    summarize(sd=sd(red/scatter),
              avg=mean(red/scatter)) %>%
    ggplot(aes(experiments, avg, group=treatment, fill=treatment)) +
    geom_bar(stat="identity", position=position_dodge()) +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +
    ylab("Red fluo (normalized to scatter)")

#### f. Orange fluorescence, normalized to scatter

In [ ]:
all %>%
    filter(population != "unknown" & population != "beads") %>%
    group_by(treatment, experiments, population) %>%
    summarize(sd=sd(orange/scatter),
              avg=mean(orange/scatter)) %>%
    ggplot(aes(experiments, avg, group=treatment, fill=treatment)) +
    geom_bar(stat="identity", position=position_dodge()) +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +
    ylab("Orange fluo (normalized to scatter)")